## Initial code written for phase field simulation 
### Code borrowed from Mao et al. Soft Matter 2019
### Set to run conserved dynamics on a phase-field model coupled to a Flory-Huggins energy

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

#==============================================================================
# packages
#==============================================================================
# ohter standard libraries
import numpy as np
import numexpr as ne
import sys
import time 
sys.path.append('Kosmrj paper/3DSim/')
from utils import convert_seconds_to_hms
import matplotlib.pyplot as plt

## FFT Solver object that does a semi-implicit-explicit calcuation for the derivative terms

In [2]:
#==============================================================================
# solver object
#==============================================================================

class FFTSolver():
    
    def __init__(self, c_init, chiMat, lmbda, dt, T, N, start, root,kappa,kon,koff):
        self.start = start
        self.root = root
        self.c0 = c_init.copy()
        self.NCom, _, _ = c_init.shape
        self.N = N
        self.gradMuX = c_init.copy()
        self.gradMuY = c_init.copy()
#         self.gradMuZ = c_init.copy()
        self.JXHat = np.zeros_like(c_init, dtype='complex')
        self.JYHat = np.zeros_like(c_init, dtype='complex')
#         self.JZHat = np.zeros_like(c_init, dtype='complex')
        self.chiMat = chiMat
        self.lmbda  = lmbda
        self.kappa = kappa
        self.dt     = dt
        self.kon    = kon
        self.koff   = koff
        self.T      = T
        self.x  = np.linspace(0, 1, N+1)[:-1]
        self.xx, self.yy = np.meshgrid(self.x, self.x)
        self.dx = 1.0/N
        self.k  = 2 * np.pi * np.fft.fftfreq(N, self.dx)
        self.kx = self.k.reshape(-1,1)
        self.ky = self.k.reshape(1,-1)
#         self.kz = self.k.reshape(1,1,-1)
        self.k2 = self.kx**2 + self.ky**2
        self.k4 = self.kx**4 + self.ky**4
        self.kxj = self.kx*1j
        self.kyj = self.ky*1j
#         self.kzj = self.kz*1j

    def fft(self, x, xHat):
        for i in range(self.NCom):
            xHat[i] = np.fft.fftn(x[i])


    def ifft(self, xHat, x):
        for i in range(self.NCom):
            x[i] = np.fft.ifftn(xHat[i]).real
        
    # calculate the chemical potential
    def cal_muHat(self, c, cHat):
        chiMat = self.chiMat
        k2 = self.k2
        lmbda = self.lmbda
        muHat = np.einsum('ij,jkl->ikl', chiMat, cHat*(1.0 - k2*lmbda))
        return muHat
    
    # calculate fluxes (multiplyied by Lij already)
    def cal_J(self, c, gradMu):
        J0 = np.einsum('ijk,ijk->jk', c, gradMu)
        J  = ne.evaluate("c * (gradMu-J0)")
        return J


    def cal_NHat(self, c, cHat, A):
        k2 = self.k2
        k4 = self.k4
        kxj = self.kxj
        kyj = self.kyj
#         kzj = self.kzj
        
        gradMuX = self.gradMuX
        gradMuY = self.gradMuY
#         gradMuZ = self.gradMuZ
        JXHat = self.JXHat
        JYHat = self.JYHat
#         JZHat = self.JZHat
        
        # diffsion part
        NDiff = - ne.evaluate("k2 * cHat")
        # implicit part
        NImp  = ne.evaluate("A * k4 * cHat")
        # fluxes part
        lmbda = self.lmbda
        chiMat = self.chiMat
#         cHatGrad = ne.evaluate("cHat*(1.0 - k2*lmbda)")
        cHatGrad = ne.evaluate("cHat*1.0")
        kappa_term = ne.evaluate("cHat*k2*lmbda")

        muHat = (chiMat.dot(cHatGrad.reshape(self.NCom, -1))).reshape(self.NCom, 
            self.N, self.N) +  (self.kappa.dot(kappa_term.reshape(self.NCom, -1))).reshape(self.NCom, 
            self.N, self.N)

        #muHat   = np.einsum('ij,jklm->iklm', chiMat, cHatGrad)
        
        # gradients -> depend on spatial dim
        muKxHat = ne.evaluate("kxj * muHat")
        self.ifft(muKxHat, gradMuX)
        self.ifft(ne.evaluate("kyj * muHat"), gradMuY)
#         self.ifft(ne.evaluate("kzj * muHat"), gradMuZ)
        # calculate fluxes:
        JX = self.cal_J(c, gradMuX)
        JY = self.cal_J(c, gradMuY)
#         JZ = self.cal_J(c, gradMuZ)
        # fluxes contribution
        self.fft(JX, JXHat)
        self.fft(JY, JYHat)
#         self.fft(JZ, JZHat)
        NFlux = ne.evaluate("kxj*JXHat + kyj*JYHat")
        return ne.evaluate("NFlux + NDiff + NImp")

    def save_images(self,c,step):
        colors = ["Greens","Oranges","Blues","Reds","Greys","Purples","PuRd","BuGn","Greens","Oranges","Blues","Reds","Greys","Purples","PuRd","BuGn"]

        levels = np.linspace(0.0, 1.0, 15)

        for i in np.arange(c.shape[0]):
            fig,ax = plt.subplots()
            cs = ax.contourf(c[i][:,:],cmap=plt.get_cmap(colors[i]),levels=levels)
            cbar = fig.colorbar(cs)
            fig.savefig(fname = self.root + 'c' + str(i) + '-' +str(self.start+step)+'.pdf',format='pdf',dpi=300)
            plt.close()
        
        print("Images saved at %d steps" % (step))
        
    def solve(self, outPrint, outSave):
        start = self.start
        root = self.root
        kon = self.kon
        koff = self.koff
        c = self.c0.copy()
        cHat = np.zeros_like(c, dtype='complex')
        self.fft(c, cHat)
        dt = self.dt
        T  = self.T
        tCur = 0.0
        A = 1.0 * self.chiMat.max() * self.lmbda
        #A = 0.0
        k4 = self.k4
        start_time = time.time()
        step = 0
        Ainv = 1.0/(1 + A * k4 * dt)
        print("--- Using %d of threads to calculate numpy fft---" % (ne.nthreads))
        while (tCur < T + dt/2.):
            tCur += dt
            step += 1
            cnHat = cHat
            cn = c
            ncHat = self.cal_NHat(cn, cnHat, A)
            cHat = ne.evaluate( "(cnHat + dt * ncHat) * Ainv" )
            self.ifft(cHat, c)
            
            c = c + ne.evaluate("(kon - koff*c)*dt")
            if np.isnan(c.max()):
                print("Simulation ended because of NAN values")
                np.save(root + 'c-'+str(start + step)+'.npy', cn)
                break
            
            if (step%outPrint == 0):
                dtime = time.time() - start_time
                print("%d steps finished with max c %.4f, using %.4f seconds/step" % (step, c.max(), dtime/step))
#                 if (start+step) < outSave:
#                     np.save(root + 'c-'+str(start+step)+'.npy', c)
#                     save_images(c,start,step,root)

            if (step%outSave == 0):
                print("----------------------step %d saved---------- ------------------" % (step+start))
                self.save_images(c,step)
                np.save(root + 'c-'+str(start+step)+'.npy', c)
                
        total_time = time.time() - start_time
        hrs, mins, secs = convert_seconds_to_hms(total_time)
        print("--- total time = %d hrs %d mins %.4f secs ---" % (hrs, mins, secs))

    



# Runtime code

### Dependencies for runtime code

In [3]:
#==============================================================================
# packages
#==============================================================================

# ohter standard libraries
import numpy as np
import os
import sys
import time
import datetime
from utils import equal_chiMat

#==============================================================================
# main_program
#==============================================================================

def makeDir(namefolder):

    if os.path.exists(namefolder):
        os.stat(namefolder)
    else:
        os.mkdir(namefolder)


In [4]:
def gen_c0(c_init, M, N,noise_strength=0.01):
    c0_init = np.array(c_init)
    cmin = c0_init.min()
    noise_initial = noise_strength * cmin * np.random.uniform(-1,1,(M, N, N))
    noise_initial -= noise_initial.mean(0)
    c0 = c0_init[:, np.newaxis, np.newaxis] + noise_initial
    return c0


### Main runtime input

In [5]:
# if __name__ == '__main__':

# sys.path.append('..')
# from utils import equal_chiMat, non_uniform_mixutre, gen_c0

###################################################
#parameters go here
###################################################

# system argument
NCom  = 3
dim   = 2
# N     = int(sys.argv[1])
# start = int(sys.argv[2])
# end   = int(sys.argv[3])
# outfolder = str(sys.argv[4])

N     = 64
start = 0
end   = 5e5
outfolder = 'Output/test_FFT_code_2D/'

steps = end - start

# physical parameters
lmbda = 5.0e-5
dt  = 2.0e-7
T   = steps * dt

chi_mean = 3.5;
chi_std =0.0;
chi = chi_std*np.random.randn(NCom,NCom) + chi_mean;
np.fill_diagonal(chi, 0);
# print(chi[-1,0:-1])
# # chi[-1,:] = 0.0;
# # print(chi[-1,0:-1])
# chi[2,1] = -4.0

g = np.zeros((NCom,NCom))

# Fill in chi and mobility matrices
np.fill_diagonal(g, (1/NCom)*(1-1/NCom))
for i in np.arange(len(chi)-1):
    for j in np.arange(i+1,len(chi)):
        chi[i,j] = chi[j,i]
        g[i,j] = -(1/NCom**2)
        g[j,i] = g[i,j]

print(chi)
wJ, vJ = np.linalg.eig(chi+np.identity(NCom)*NCom)
print(min(wJ))
Jeff = np.matmul(g,chi+np.identity(NCom)*NCom);
wJeff,vJeff = np.linalg.eig(Jeff)
print(min(wJeff))

kon = 0.0;
koff = kon*NCom;


chiMat = chi;
kappa = np.identity(NCom)*10.0
# chi01 = 3.00
# chi02 = 7.00
# chi12 = 4.00
# chi03 = 6.00

# # ternary
# chiMat[0, 1] = chiMat[1, 0] = chi01
# chiMat[1, 2] = chiMat[2, 1] = chi12
# chiMat[0, 2] = chiMat[2, 0] = chi02
# # the fourth component
# chiMat[0, 3] = chiMat[3, 0] = chi03
# chiMat[1, 3] = chiMat[3, 1] = chi01
# chiMat[2, 3] = chiMat[3, 2] = chi02
# # 5-comps
# chiMat[0, 4] = chiMat[4, 0] = chi03
# chiMat[1, 4] = chiMat[4, 1] = chi01
# chiMat[2, 4] = chiMat[4, 2] = chi02
# chiMat[3, 4] = chiMat[4, 3] = chi03

# output control
outPrint = 25000
outSave  = 25000

random_seed = np.random.randint(1e6)


###################################################
#initialize
###################################################

if start < 1:
    c_init = np.ones((NCom))/NCom
    #c0 = non_uniform_mixutre(c_init, N=N, dim=dim)
    c0 = gen_c0(c_init, NCom, N)
else:
    c0 = np.load(root+'c-'+str(start)+'.npy')

print(c0.shape)

[[0.  3.5 3.5]
 [3.5 0.  3.5]
 [3.5 3.5 0. ]]
-0.49999999999999983
-0.16666666666666666
(3, 64, 64)


In [6]:
end = 5e6
root =  outfolder + '/' + str(datetime.date.today()).replace('-','') + '/' + str(N) + '_' + str(start) + '_' + str(end) + '_' + str(NCom) + '_' + str(kon) + '/' + str(random_seed) + '/'
os.makedirs(root,exist_ok=True)
print(root)
np.save(root+'c-'+str(start)+'.npy', c0)
###################################################
#solving
###################################################

# from FFT_nV_3D import FFTSolver

Solver1 = FFTSolver(c_init=c0,
                    chiMat=chiMat, lmbda=lmbda,
                    dt=dt, T=T, N=N,
                    start=start, root=root,kappa=kappa,kon=kon,koff=koff)

Solver1.save_images(c0,start)
cFFT3 = Solver1.solve(outPrint=outPrint, outSave=outSave)
#np.save(root+'c-'+str(end + 1)+'.npy', cFFT3)

###################################################
#end
###################################################


Output/test_FFT_code_2D//20210208/64_0_5000000.0_3_0.0/84548/
Images saved at 0 steps
--- Using 8 of threads to calculate numpy fft---
25000 steps finished with max c 0.3345, using 0.0024 seconds/step
----------------------step 25000 saved---------- ------------------
Images saved at 25000 steps
50000 steps finished with max c 0.3346, using 0.0024 seconds/step
----------------------step 50000 saved---------- ------------------
Images saved at 50000 steps
75000 steps finished with max c 0.3347, using 0.0024 seconds/step
----------------------step 75000 saved---------- ------------------
Images saved at 75000 steps
100000 steps finished with max c 0.3349, using 0.0024 seconds/step
----------------------step 100000 saved---------- ------------------
Images saved at 100000 steps
125000 steps finished with max c 0.3351, using 0.0024 seconds/step
----------------------step 125000 saved---------- ------------------
Images saved at 125000 steps
150000 steps finished with max c 0.3354, using 0.

In [7]:
    # sim_output_files = os.listdir(root)
    # output_data = []
    # for f in sim_output_files:
    #     if f.find('.npy') > -1:
    #         output_data.append(np.load(root+f))
    # print(len(output_data))

In [8]:
# od = output_data[-1].reshape((6,64*64))
# od.shape

## import matplotlib.pyplot as plt
# for i in np.arange(len(output_data)):
#     fig,ax = plt.subplots()
#     cs = ax.contourf(output_data[i][0][:,:,10],cmap=plt.get_cmap("Greens"),vmin = 0,vmax =1.0)
#     cbar = fig.colorbar(cs)
# plt.show()

J = chi + np.identity(NCom)*NCom
print(J)
print(np.linalg.eigvals(J))
Jeff = np.matmul(g,J)
print(np.linalg.eigvals(Jeff))
Jeff = np.matmul(J,g)
print(np.linalg.eigvals(Jeff))
Jeff = np.matmul(g,chi+np.identity(NCom)*NCom);
wJeff,vJeff = np.linalg.eig(Jeff)
print(wJeff)